In [2]:
import pandas as pd
import pickle
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# 데이터 로딩
with open('clustering_result_project.pkl', 'rb') as f:
    df = pickle.load(f)

df_cluster1 = df[df['cluster']==1].copy()
print(f"원본 Cluster 1 데이터 수: {len(df_cluster1)}")

# 속도 향상을 위한 데이터 크기 조정 (필요시)
if len(df_cluster1) > 3000:
    print("데이터가 많아 샘플링을 진행합니다...")
    df_cluster1 = df_cluster1.sample(3000, random_state=42)
    print(f"샘플링 후 데이터 수: {len(df_cluster1)}")

import gensim
from gensim import corpora, models
from gensim.corpora import Dictionary
import numpy as np

# 문서 준비
all_documents = list(df_cluster1['review_words'])
print("사전 생성 중...")

# 사전 생성 및 필터링 (속도 향상)
dictionary = Dictionary(all_documents)
print(f"필터링 전 사전 크기: {len(dictionary)}")

# 극단적으로 빈번하거나 희귀한 단어 제거
dictionary.filter_extremes(no_below=3, no_above=0.7, keep_n=1500)
print(f"필터링 후 사전 크기: {len(dictionary)}")

# 코퍼스 생성
print("코퍼스 생성 중...")
corpus = [dictionary.doc2bow(doc) for doc in all_documents]
print(f"코퍼스 크기: {len(corpus)}")

# 빠른 LDA 모델 학습
topic_num = 3
print(f"\n{topic_num}개 토픽으로 LDA 모델 학습 시작...")

ldamodel = gensim.models.ldamodel.LdaModel(
    corpus,
    num_topics=topic_num,
    id2word=dictionary,
    passes=10,           # 원래 20에서 10으로 줄임
    iterations=25,       # 원래 50에서 25로 줄임
    random_state=42,
    alpha='auto',        # 자동 최적화로 빠른 수렴
    eta='auto',          # 베타도 자동 최적화
    per_word_topics=True,
    eval_every=5         # 평가 주기 늘려서 속도 향상
)

print("LDA 모델 학습 완료!")

# 토픽 결과 출력
print("\n=== 토픽 결과 ===")
topics = ldamodel.show_topics(num_topics=-1, formatted=True)
for topic_id, topic_desc in topics:
    print(f"토픽 {topic_id + 1}: {topic_desc}")

# 성능 평가 (선택사항 - 빠르게 하려면 주석 처리 가능)
from gensim.models import CoherenceModel

try:
    print("\nCoherence 점수 계산 중...")
    coherence_model = CoherenceModel(
        model=ldamodel,
        texts=all_documents,
        dictionary=dictionary,
        coherence='c_v'
    )
    coherence_score = coherence_model.get_coherence()
    print(f"Coherence 점수: {coherence_score:.4f}")
except:
    print("Coherence 계산을 건너뜁니다.")

# 문서별 토픽 할당
print("\n문서별 토픽 할당 중...")
action_align = []
doc_topics = ldamodel.get_document_topics(corpus)

for doc in tqdm(doc_topics):
    if doc:
        dominant_topic = max(doc, key=lambda x: x[1])[0]
    else:
        dominant_topic = 0
    action_align.append(dominant_topic + 1)  # 1부터 시작

df_cluster1['action_cluster'] = action_align

print("토픽 분포:")
print(df_cluster1['action_cluster'].value_counts().sort_index())

# ================================
# pyLDAvis 시각화 (필수 부분)
# ================================
print("\n=== pyLDAvis 시각화 생성 중 ===")

try:
    import pyLDAvis.gensim_models as gensimvis
    import pyLDAvis
    
    # 경고 무시
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    
    print("시각화 데이터 준비 중...")
    prepared_data = gensimvis.prepare(ldamodel, corpus, dictionary)
    
    print("HTML 파일 저장 중...")
    pyLDAvis.save_html(prepared_data, 'ldavis_cluster1.html')
    print("✅ 'ldavis_cluster1.html' 파일이 저장되었습니다!")
    
    # 주피터 노트북에서 실행 중인 경우 표시
    print("시각화 표시 중...")
    pyLDAvis.display(prepared_data)
    
except ImportError:
    print("❌ pyLDAvis가 설치되지 않았습니다.")
    print("다음 명령어로 설치하세요: pip install pyLDAvis")
except Exception as e:
    print(f"❌ 시각화 생성 중 오류 발생: {e}")
    print("모델은 정상적으로 생성되었지만 시각화에 문제가 있습니다.")

# 결과 저장
print("\n=== 결과 저장 중 ===")
df_cluster1_final = df_cluster1[['review', 're_review', 'review_words', 'action_cluster']].copy()

df_cluster1_final.to_csv('cluster_project_1_1st.csv', encoding='utf-8-sig', index=False)
print("✅ 'cluster_project_1_1st.csv' 저장 완료")

with open('Cluster1.pkl', 'wb') as f:
    pickle.dump(df_cluster1_final, f)
print("✅ 'Cluster1.pkl' 저장 완료")

# 최종 요약
print(f"\n=== 최종 결과 ===")
print(f"처리된 데이터 수: {len(df_cluster1_final)}")
print(f"생성된 토픽 수: {topic_num}")
print("토픽 분포:")
for topic_num, count in df_cluster1_final['action_cluster'].value_counts().sort_index().items():
    print(f"  토픽 {topic_num}: {count}개 문서")
print(f"저장된 파일: cluster_project_1_1st.csv, Cluster1.pkl, ldavis_cluster1.html")

원본 Cluster 1 데이터 수: 520
사전 생성 중...
필터링 전 사전 크기: 14409
필터링 후 사전 크기: 1500
코퍼스 생성 중...
코퍼스 크기: 520

3개 토픽으로 LDA 모델 학습 시작...
LDA 모델 학습 완료!

=== 토픽 결과 ===
토픽 1: 0.020*"생각" + 0.014*"적금" + 0.014*"아니다" + 0.013*"좋다" + 0.012*"그렇다" + 0.008*"친구" + 0.008*"많다" + 0.007*"월급" + 0.006*"정말" + 0.006*"그냥"
토픽 2: 0.017*"맛있다" + 0.015*"좋다" + 0.011*"사진" + 0.010*"아니다" + 0.008*"엄마" + 0.008*"저녁" + 0.008*"그렇다" + 0.007*"많다" + 0.007*"아침" + 0.007*"귀엽다"
토픽 3: 0.021*"좋다" + 0.016*"사람" + 0.013*"아니다" + 0.010*"공부" + 0.009*"그냥" + 0.009*"그렇다" + 0.008*"많다" + 0.008*"생각" + 0.008*"시험" + 0.007*"시작"

Coherence 점수 계산 중...
Coherence 점수: 0.2749

문서별 토픽 할당 중...


100%|██████████| 520/520 [00:00<00:00, 1008.70it/s]


토픽 분포:
action_cluster
1    152
2    210
3    158
Name: count, dtype: int64

=== pyLDAvis 시각화 생성 중 ===
시각화 데이터 준비 중...
HTML 파일 저장 중...
✅ 'ldavis_cluster1.html' 파일이 저장되었습니다!
시각화 표시 중...

=== 결과 저장 중 ===
✅ 'cluster_project_1_1st.csv' 저장 완료
✅ 'Cluster1.pkl' 저장 완료

=== 최종 결과 ===
처리된 데이터 수: 520
생성된 토픽 수: 3
토픽 분포:
  토픽 1: 152개 문서
  토픽 2: 210개 문서
  토픽 3: 158개 문서
저장된 파일: cluster_project_1_1st.csv, Cluster1.pkl, ldavis_cluster1.html


In [3]:
prepared_data = gensimvis.prepare(ldamodel, corpus, dictionary)
pyLDAvis.save_html(prepared_data, 'ldavis_cluster1.html')
pyLDAvis.display(prepared_data)

In [4]:
import pandas as pd
import pickle
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# 데이터 로딩
with open('clustering_result_project.pkl', 'rb') as f:
    df = pickle.load(f)

df_cluster1_2 = df[df['cluster']==1].copy()
print(f"원본 Cluster 1 데이터 수: {len(df_cluster1_2)}")

# 속도 향상을 위한 데이터 크기 조정 (필요시)
if len(df_cluster1_2) > 3000:
    print("데이터가 많아 샘플링을 진행합니다...")
    df_cluster1 = df_cluster1.sample(3000, random_state=42)
    print(f"샘플링 후 데이터 수: {len(df_cluster1_2)}")

import gensim
from gensim import corpora, models
from gensim.corpora import Dictionary
import numpy as np

# 문서 준비
all_documents = list(df_cluster1_2['review_words'])
print("사전 생성 중...")

# 사전 생성 및 필터링 (속도 향상)
dictionary = Dictionary(all_documents)
print(f"필터링 전 사전 크기: {len(dictionary)}")

# 극단적으로 빈번하거나 희귀한 단어 제거
dictionary.filter_extremes(no_below=3, no_above=0.7, keep_n=1500)
print(f"필터링 후 사전 크기: {len(dictionary)}")

# 코퍼스 생성
print("코퍼스 생성 중...")
corpus = [dictionary.doc2bow(doc) for doc in all_documents]
print(f"코퍼스 크기: {len(corpus)}")

# 빠른 LDA 모델 학습
topic_num = 4
print(f"\n{topic_num}개 토픽으로 LDA 모델 학습 시작...")

ldamodel = gensim.models.ldamodel.LdaModel(
    corpus,
    num_topics=topic_num,
    id2word=dictionary,
    passes=10,           # 원래 20에서 10으로 줄임
    iterations=25,       # 원래 50에서 25로 줄임
    random_state=42,
    alpha='auto',        # 자동 최적화로 빠른 수렴
    eta='auto',          # 베타도 자동 최적화
    per_word_topics=True,
    eval_every=5         # 평가 주기 늘려서 속도 향상
)

print("LDA 모델 학습 완료!")

# 토픽 결과 출력
print("\n=== 토픽 결과 ===")
topics = ldamodel.show_topics(num_topics=-1, formatted=True)
for topic_id, topic_desc in topics:
    print(f"토픽 {topic_id + 1}: {topic_desc}")

# 성능 평가 (선택사항 - 빠르게 하려면 주석 처리 가능)
from gensim.models import CoherenceModel

try:
    print("\nCoherence 점수 계산 중...")
    coherence_model = CoherenceModel(
        model=ldamodel,
        texts=all_documents,
        dictionary=dictionary,
        coherence='c_v'
    )
    coherence_score = coherence_model.get_coherence()
    print(f"Coherence 점수: {coherence_score:.4f}")
except:
    print("Coherence 계산을 건너뜁니다.")

# 문서별 토픽 할당
print("\n문서별 토픽 할당 중...")
action_align = []
doc_topics = ldamodel.get_document_topics(corpus)

for doc in tqdm(doc_topics):
    if doc:
        dominant_topic = max(doc, key=lambda x: x[1])[0]
    else:
        dominant_topic = 0
    action_align.append(dominant_topic + 1)  # 1부터 시작

df_cluster1_2['action_cluster'] = action_align

print("토픽 분포:")
print(df_cluster1_2['action_cluster'].value_counts().sort_index())

# ================================
# pyLDAvis 시각화 (필수 부분)
# ================================
print("\n=== pyLDAvis 시각화 생성 중 ===")

try:
    import pyLDAvis.gensim_models as gensimvis
    import pyLDAvis
    
    # 경고 무시
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    
    print("시각화 데이터 준비 중...")
    prepared_data = gensimvis.prepare(ldamodel, corpus, dictionary)
    
    print("HTML 파일 저장 중...")
    pyLDAvis.save_html(prepared_data, 'ldavis_cluster1_2.html')
    print("✅ 'ldavis_cluster1_2.html' 파일이 저장되었습니다!")
    
    # 주피터 노트북에서 실행 중인 경우 표시
    print("시각화 표시 중...")
    pyLDAvis.display(prepared_data)
    
except ImportError:
    print("❌ pyLDAvis가 설치되지 않았습니다.")
    print("다음 명령어로 설치하세요: pip install pyLDAvis")
except Exception as e:
    print(f"❌ 시각화 생성 중 오류 발생: {e}")
    print("모델은 정상적으로 생성되었지만 시각화에 문제가 있습니다.")

# 결과 저장
print("\n=== 결과 저장 중 ===")
df_cluster1_final_2 = df_cluster1[['review', 're_review', 'review_words', 'action_cluster']].copy()

df_cluster1_final_2.to_csv('cluster_project_1_2.csv', encoding='utf-8-sig', index=False)
print("✅ 'cluster_project_1_2t.csv' 저장 완료")

with open('Cluster1_2.pkl', 'wb') as f:
    pickle.dump(df_cluster1_final_2, f)
print("✅ 'Cluster1_2.pkl' 저장 완료")

# 최종 요약
print(f"\n=== 최종 결과 ===")
print(f"처리된 데이터 수: {len(df_cluster1_final)}")
print(f"생성된 토픽 수: {topic_num}")
print("토픽 분포:")
for topic_num, count in df_cluster1_final['action_cluster'].value_counts().sort_index().items():
    print(f"  토픽 {topic_num}: {count}개 문서")
print(f"저장된 파일: cluster_project_1_2.csv, Cluster_2.pkl, ldavis_cluster_2.html")

원본 Cluster 1 데이터 수: 520
사전 생성 중...
필터링 전 사전 크기: 14409
필터링 후 사전 크기: 1500
코퍼스 생성 중...
코퍼스 크기: 520

4개 토픽으로 LDA 모델 학습 시작...
LDA 모델 학습 완료!

=== 토픽 결과 ===
토픽 1: 0.024*"생각" + 0.015*"아니다" + 0.014*"그렇다" + 0.014*"좋다" + 0.012*"적금" + 0.009*"친구" + 0.007*"월급" + 0.007*"사람" + 0.007*"그냥" + 0.006*"사실"
토픽 2: 0.018*"맛있다" + 0.014*"좋다" + 0.012*"사진" + 0.011*"아니다" + 0.009*"저녁" + 0.009*"엄마" + 0.008*"그렇다" + 0.008*"아침" + 0.007*"점심" + 0.007*"많다"
토픽 3: 0.019*"좋다" + 0.016*"사람" + 0.012*"아니다" + 0.011*"공부" + 0.010*"그냥" + 0.009*"적금" + 0.009*"만원" + 0.009*"시험" + 0.008*"그렇다" + 0.008*"생각"
토픽 4: 0.018*"좋다" + 0.011*"아니다" + 0.011*"많다" + 0.010*"맛있다" + 0.007*"적금" + 0.006*"보고" + 0.006*"사람" + 0.006*"새롭다" + 0.006*"이제" + 0.005*"생각"

Coherence 점수 계산 중...
Coherence 점수: 0.2652

문서별 토픽 할당 중...


100%|██████████| 520/520 [00:00<00:00, 5121.17it/s]

토픽 분포:
action_cluster
1    126
2    173
3    120
4    101
Name: count, dtype: int64

=== pyLDAvis 시각화 생성 중 ===
시각화 데이터 준비 중...


HTML 파일 저장 중...
✅ 'ldavis_cluster1_2.html' 파일이 저장되었습니다!
시각화 표시 중...

=== 결과 저장 중 ===
✅ 'cluster_project_1_2t.csv' 저장 완료
✅ 'Cluster1_2.pkl' 저장 완료

=== 최종 결과 ===
처리된 데이터 수: 520
생성된 토픽 수: 4
토픽 분포:
  토픽 1: 152개 문서
  토픽 2: 210개 문서
  토픽 3: 158개 문서
저장된 파일: cluster_project_1_2.csv, Cluster_2.pkl, ldavis_cluster_2.html


In [5]:
prepared_data = gensimvis.prepare(ldamodel, corpus, dictionary)
pyLDAvis.save_html(prepared_data, 'ldavis_cluster1_2.html')
pyLDAvis.display(prepared_data)